In [3]:
# start with importing the libraries we need
%matplotlib inline
import numpy as np
import brian2 as b2
import matplotlib.pyplot as plt
from neurodynex3.leaky_integrate_and_fire import LIF
from neurodynex3.tools import input_factory, plot_tools

In [ ]:
#LIF.getting_started()

In [4]:
# Define the parameter
LIF.V_REST = 15*b2.mV
LIF.V_RESET = -50*b2.mV
LIF.FIRING_THRESHOLD = 0*b2.mV
LIF.MEMBRANE_RESISTANCE = 50. * b2.ohm
LIF.MEMBRANE_TIME_SCALE = 10. * b2.ms
ABSOLUTE_REFRACTORY_PERIOD = 0. * b2.ms ####### not sure we have to change this parameter (initially 2.0 * b2.ms) 
beta = 10e5 #s^-1 -> have to precise the factor beta for the instantaneous firing rate

LIF.print_default_parameters()

Resting potential: 0.015
Reset voltage: -0.05
Firing threshold: 0.0
Membrane resistance: 50.0
Membrane time-scale: 0.01
Absolute refractory period: 0.002


1. Consider N = 100 non-interacting neurons, with initial voltages $v_i(t = 0)$ uniformly distributed at random between $v_0$ and $η$ (i.e. $v_i(0) ∼ U([v_0,η])$), and submitted to an oscillating external input $I_{ext}(t) = I_0(1 + sin(ωt))$, with $I_0 = 0.5 mA$, $ω = 10^2 s^{−1}(= 0.1 ms^{−1})$. <br>
• Compute the membrane potentials $vi(t)$ and firing times of the neurons, for $t ≤ T_f = 100 ms$. <br>
• Plot the voltage $v_i(t)$ of the first neuron, along with the external input $RI_{ext}(t)$.

In [ ]:
N = 100
vi_0 = np.random.uniform(v0, eta, N)
I0 = 0.5e-3 #A
omega = 10e2 #s-1
t = 
Iext = I0 * (1 + np.sin(omega * np.arange(0, 1, 1e-3))